In [178]:
import requests

token = 'cd5b792d5b2d7a65e9c31fd710773253f2772f5d'


In [179]:
#search request to get all Mumbai stations
URL = "http://api.waqi.info/search/?keyword=Mumbai&token=cd5b792d5b2d7a65e9c31fd710773253f2772f5d"
r = requests.get(url = URL)
data = r.json()


In [180]:
#create an array of stations
stations = [item['station'] for item in data['data']]
#create an array of station URLs to be incorporated into each station-specific request
urls = [ item['url'] for item in stations ]
urls

['india/mumbai/sion',
 'india/mumbai/bandra-kurla-complex',
 'india/mumbai/worli',
 'india/mumbai/powai',
 'india/mumbai/siddharth-nagar-worli',
 'india/mumbai/us-consulate',
 'india/mumbai/chhatrapati-shivaji-intl.-airport-t2',
 'india/mumbai/vile-parle-west',
 'india/mumbai/kurla',
 'india/mumbai/deonar',
 'india/mumbai/chakala-andheri-east',
 'india/mumbai/bandra',
 'india/mumbai/colaba',
 'india/mumbai/vasai-west',
 'india/navi-mumbai/nerul',
 'india/navi-mumbai/mahape',
 'india/mumbai/nmmc-airoli',
 'india/mumbai/borivali-east',
 'india/navi-mumbai/sector-19a-nerul']

In [230]:
#get current data based on each URL
stations_data = []
for item in urls:
  url = "https://api.waqi.info/feed/{url}/?token={token}".format(token=token, url=item)
  r = requests.get(url = url)
  data = r.json()
  stations_data.append(data)
  

In [256]:
#example for one station data
stations_data[0]['data']

{'aqi': 196,
 'idx': 12464,
 'attributions': [{'url': 'http://cpcb.nic.in/',
   'name': 'CPCB - India Central Pollution Control Board',
   'logo': 'India-CPCB.png'},
  {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
 'city': {'geo': [19.047, 72.8746],
  'name': 'Sion, Mumbai, India',
  'url': 'https://aqicn.org/city/india/mumbai/sion',
  'location': ''},
 'dominentpol': 'pm25',
 'iaqi': {'co': {'v': 10.4},
  'dew': {'v': 13},
  'h': {'v': 33.73},
  'no2': {'v': 4.9},
  'o3': {'v': 5.6},
  'p': {'v': 1009},
  'pm10': {'v': 102},
  'pm25': {'v': 196},
  'so2': {'v': 7.2},
  't': {'v': 37},
  'w': {'v': 5.4375},
  'wg': {'v': 10.8}},
 'time': {'s': '2023-03-11 14:00:00',
  'tz': '+05:30',
  'v': 1678543200,
  'iso': '2023-03-11T14:00:00+05:30'},
 'forecast': {'daily': {'o3': [{'avg': 20,
     'day': '2023-03-09',
     'max': 88,
     'min': 1},
    {'avg': 19, 'day': '2023-03-10', 'max': 58, 'min': 1},
    {'avg': 21, 'day': '2023-03-11', 'max': 73, 'min': 2},
 

In [271]:
import pandas as pd

df = pd.DataFrame()

data_ar = []

indicators = ['co', 'no2', 'so2', 'o3', 'pm10', 'pm25']

for station_data in stations_data:
    aq = station_data['data']['iaqi']
    aqi = station_data['data']['aqi']
    row = {}
    row.update({'station':station_data['data']['city']['name']})
    row.update({'time':station_data['data']['time']['s']})
    #assign non existent values with None
    for ind in indicators:
        if aq[ind] != None:
            row.update({ind:aq[ind]['v']})
    data_ar.append(row)

In [272]:
pd.DataFrame(data_ar)

,station,time,co,no2,so2,o3,pm10,pm25
0,"Sion, Mumbai, India",2023-03-11 14:00:00,10.4,4.9,7.2,5.6,102.0,196.0
1,"Bandra Kurla Complex, Mumbai, India",2023-03-11 14:00:00,2.2,68.2,30.7,77.3,120.0,168.0
2,"Worli, Mumbai, India",2023-03-11 15:00:00,15.6,13.4,2.5,0.4,60.0,129.0
3,"Powai, Mumbai, India",2023-03-11 15:00:00,5.8,17.4,6.7,22.0,82.0,149.0
4,"Siddharth Nagar-Worli, Mumbai, India",2023-03-11 15:00:00,5.7,8.4,NaN,67.1,76.0,148.0
5,"Mumbai US Consulate, India (मुंबई अमेरिकी वाणि...",2023-03-11 15:00:00,NaN,NaN,NaN,NaN,NaN,122.0
6,"Chhatrapati Shivaji Intl. Airport (T2), Mumbai...",2023-03-11 15:00:00,15.3,47.3,4.2,9.7,97.0,101.0
7,"Vile Parle West, Mumbai, India",2023-03-11 15:00:00,11.1,10.7,29.8,13.7,92.0,NaN
8,"Kurla, Mumbai, India",2023-03-11 14:00:00,3.3,8.3,13.0,56.5,78.0,70.0
9,"Deonar, Mumbai, India",2023-03-03 20:00:00,16.0,28.5,4.1,11.2,163.0,228.0
